In [1]:
library(readr)
library(survival)
library(dplyr)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [2]:
TENURES <- read_csv('categorized/tenures-summarized-categorized.csv', na=character())

Parsed with column specification:
cols(
  slug = col_character(),
  coach = col_character(),
  coach_id = col_character(),
  franchise = col_character(),
  seasons = col_integer(),
  games = col_integer(),
  wins = col_integer(),
  losses = col_integer(),
  min_date = col_date(format = ""),
  max_date = col_date(format = ""),
  left_truncated = col_character(),
  ending = col_character()
)


In [3]:
head(TENURES)

slug,coach,coach_id,franchise,seasons,games,wins,losses,min_date,max_date,left_truncated,ending
krugelo99c|00|02|ATL,Lon Kruger,krugelo99c,ATL,3,191,69,122,2000-10-31,2002-12-23,True,fired
stottte99c|02|04|ATL,Terry Stotts,stottte99c,ATL,2,137,52,85,2002-12-27,2004-04-14,False,fired
woodsmi01c|04|10|ATL,Mike Woodson,woodsmi01c,ATL,6,492,206,286,2004-11-03,2010-04-14,False,fired
drewla01c|10|13|ATL,Larry Drew,drewla01c,ATL,3,230,128,102,2010-10-27,2013-04-17,False,fired
budenmi99c|13|17|ATL,Mike Budenholzer,budenmi99c,ATL,4,328,189,139,2013-10-30,2017-04-12,False,
pitinri99c|00|01|BOS,Rick Pitino,pitinri99c,BOS,1,34,12,22,2000-11-01,2001-01-06,True,resigned


In [4]:
TENURES %>%
group_by(ending) %>%
summarize(ending_freq = n()) %>%
arrange(desc(ending_freq))

ending,ending_freq
fired,112
interim only,39
,30
resigned,26
contract expired,2
mutual,2
retired,2
traded,2
died,1


In [5]:
tenures <- TENURES %>% filter(ending != 'interim only')

In [6]:
death_event <- c('fired', 'contract expired', 'mutual')

In [7]:
tenures <- tenures %>% mutate(death = (ending %in% death_event)*1)

In [8]:
tenures %>% head

slug,coach,coach_id,franchise,seasons,games,wins,losses,min_date,max_date,left_truncated,ending,death
krugelo99c|00|02|ATL,Lon Kruger,krugelo99c,ATL,3,191,69,122,2000-10-31,2002-12-23,True,fired,1
stottte99c|02|04|ATL,Terry Stotts,stottte99c,ATL,2,137,52,85,2002-12-27,2004-04-14,False,fired,1
woodsmi01c|04|10|ATL,Mike Woodson,woodsmi01c,ATL,6,492,206,286,2004-11-03,2010-04-14,False,fired,1
drewla01c|10|13|ATL,Larry Drew,drewla01c,ATL,3,230,128,102,2010-10-27,2013-04-17,False,fired,1
budenmi99c|13|17|ATL,Mike Budenholzer,budenmi99c,ATL,4,328,189,139,2013-10-30,2017-04-12,False,,0
pitinri99c|00|01|BOS,Rick Pitino,pitinri99c,BOS,1,34,12,22,2000-11-01,2001-01-06,True,resigned,0


In [9]:
POC <- read_csv('categorized//coach-list-poc.csv')

Parsed with column specification:
cols(
  coach = col_character(),
  coach_id = col_character(),
  franchises = col_character(),
  poc = col_character(),
  note = col_character()
)


In [10]:
head(POC)

coach,coach_id,franchises,poc,note
Tyronn Lue,luety01c,CLE,yes,NA
Tyrone Corbin,corbity01c,SAC|UTA,yes,NA
Tony Brown,brownto01c,NJNBRK,yes,NA
Terry Porter,portete01c,PHO|MIL,yes,NA
Sidney Lowe,lowesi01c,VANMEM,yes,NA
Sam Vincent,vincesa01c,CHARBH,yes,NA


In [11]:
PLAYERS <- read_csv('categorized/coach-list-qualifications.csv')

Parsed with column specification:
cols(
  coach = col_character(),
  coach_id = col_character(),
  franchises = col_character(),
  former_nba_player = col_character()
)


In [12]:
head(PLAYERS)

coach,coach_id,franchises,former_nba_player
Isiah Thomas,thomais01c,IND|NYK,yes
Michael Cooper,coopemi01c,DEN,yes
Larry Drew,drewla01c,ATL|MIL,yes
Vinny Del Negro,delnevi01c,CHI|LAC,yes
Paul Westphal,westppa01c,SAC|SEAOKC,yes
Mike Woodson,woodsmi01c,NYK|ATL,yes


In [13]:
tenures_info <- list(
    tenures,
    POC %>% select(coach_id, poc),
    PLAYERS %>% select(coach_id, former_nba_player)
) %>%
Reduce(function(dtf1,dtf2) left_join(dtf1,dtf2,by="coach_id"), .)

In [14]:
head(tenures_info)

slug,coach,coach_id,franchise,seasons,games,wins,losses,min_date,max_date,left_truncated,ending,death,poc,former_nba_player
krugelo99c|00|02|ATL,Lon Kruger,krugelo99c,ATL,3,191,69,122,2000-10-31,2002-12-23,True,fired,1,no,no
stottte99c|02|04|ATL,Terry Stotts,stottte99c,ATL,2,137,52,85,2002-12-27,2004-04-14,False,fired,1,no,no
woodsmi01c|04|10|ATL,Mike Woodson,woodsmi01c,ATL,6,492,206,286,2004-11-03,2010-04-14,False,fired,1,yes,yes
drewla01c|10|13|ATL,Larry Drew,drewla01c,ATL,3,230,128,102,2010-10-27,2013-04-17,False,fired,1,yes,yes
budenmi99c|13|17|ATL,Mike Budenholzer,budenmi99c,ATL,4,328,189,139,2013-10-30,2017-04-12,False,,0,no,no
pitinri99c|00|01|BOS,Rick Pitino,pitinri99c,BOS,1,34,12,22,2000-11-01,2001-01-06,True,resigned,0,no,no


In [15]:
surv.all <- coxph(Surv(games, death) ~ poc + former_nba_player, data = tenures_info)

In [16]:
surv.all

Call:
coxph(formula = Surv(games, death) ~ poc + former_nba_player, 
    data = tenures_info)

                       coef exp(coef) se(coef)    z     p
pocyes               0.3466    1.4143   0.1918 1.81 0.071
former_nba_playeryes 0.0528    1.0543   0.2062 0.26 0.798

Likelihood ratio test=3.65  on 2 df, p=0.161
n= 176, number of events= 115 
   (1 observation deleted due to missingness)